written by Zetian

extract top key skills in jobss_cleaned.csv

In [4]:
# Written by Zetian
import re
from collections import Counter
import pandas as pd
import numpy as np

input_path = "/Users/sky/Library/CloudStorage/GoogleDrive-skyjin0127@gmail.com/其他计算机/我的笔记本电脑/jzt/BU课程/Spring_2025/CS506/project/cs506-project/datasets/processed_data/jobss_cleaned.csv"             
output_path = "industry_top_skill_keywords.csv"   
top_n = 50                                       

df = pd.read_csv(input_path)
df.dropna(subset=["Industry", "Key Skills"], inplace=True)

# ———— 2. 按 Industry（Category） 聚合并统计技能频次 ————
industry_skill_counts = {}

for industry, group in df.groupby("Industry"):
    counter = Counter()
    for skills in group["Key Skills"]:
        # 用正则按 “|” 分割，并去掉多余空白
        tokens = re.split(r"\s*\|\s*", skills)
        for token in tokens:
            token = token.strip()
            if token:
                counter[token] += 1
    # 将频次转换为 numpy int，并取出现频次最高的 top_n 个 (keyword, np.int) 对
    top_list = [(skill, np.int64(freq)) for skill, freq in counter.most_common(top_n)]
    industry_skill_counts[industry] = top_list

# ———— 3. 构建结果 DataFrame ————
result_rows = []
for industry, top_list in industry_skill_counts.items():
    result_rows.append({
        "Category": industry,
        "top_keywords": top_list  # 列表中的元素形如 (keyword, np.int64)
    })

result_df = pd.DataFrame(result_rows)

# ———— 4. 保存到 CSV ————
# 注意：带列表/元组的列会以字符串形式写入 CSV，如需复原可用 ast.literal_eval
result_df.to_csv(output_path, index=False)